In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing, utils
import re
import os
import yaml

### Criar uma lista com todos os arquivos

In [2]:
dir_path = '../../../dados/portuguese/'
files_list = os.listdir(dir_path + os.sep)

In [3]:
questions = []
answers = []

for filepath in files_list:
    file = open(dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(file)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 1 :
            questions.append(con[0])
            answers.append(con[1])

In [4]:
answers_tags = []
for i in range(len(answers)) :
    answers_tags.append('<START> ' + answers[i] + ' <END>')


In [5]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers_tags)
VOCAB_SIZE = len(tokenizer.word_index)+1

In [6]:
answers_tags

['<START> eu sei <END>',
 '<START> também <END>',
 '<START> que bom <END>',
 '<START> obrigado <END>',
 '<START> você é que é <END>',
 '<START> que bom <END>',
 '<START> sim, eu sei <END>',
 '<START> nem tanto <END>',
 '<START> é bom para saúde ser animada <END>',
 '<START> ninguém é insubstituível <END>',
 '<START> você é que trouxe luz a minha vida <END>',
 '<START> eu te amo por completo <END>',
 '<START> que bom, obrigada <END>',
 '<START> obrigada <END>',
 '<START> que bom, obrigado <END>',
 '<START> obrigado <END>',
 '<START> você acha? <END>',
 '<START> que bom <END>',
 '<START> que bom <END>',
 '<START> obrigada <END>',
 '<START> você é que está <END>',
 '<START> obrigado <END>',
 '<START> obrigada <END>',
 '<START> estudo para isso <END>',
 '<START> é sim <END>',
 '<START> você também <END>',
 '<START> você também é maravilhosa <END>',
 '<START> obrigada <END>',
 '<START> você também fala bem <END>',
 '<START> tenho sim <END>',
 '<START> obrigado <END>',
 '<START> obrigada <EN

In [7]:
VOCAB_SIZE

1220

In [8]:
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
encoder_input_data = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')

In [9]:
print(encoder_input_data.shape)

(280, 28)


In [10]:
tokenized_answers = tokenizer.texts_to_sequences(answers_tags)

In [11]:
maxlen_answers = max([len(x) for x in tokenized_answers])
decoder_input_data = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')

In [12]:
print(decoder_input_data.shape)

(280, 47)


In [13]:
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_output_data = utils.to_categorical(padded_answers , VOCAB_SIZE)

In [14]:
tokenized_answers[10]

[7, 4, 6, 282, 133, 9, 45, 68, 2]

In [15]:
print(decoder_output_data.shape)

(280, 47, 1220)


In [16]:
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions ,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

In [17]:
decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers , ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)

In [18]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 47)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 28, 200)              244000    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 47, 200)              244000    ['input_2[0][0]']             
                                                                                              

In [20]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=32, epochs=200) 

Epoch 1/200
9/9 [==============================] - 11s 207ms/step - loss: 7.0857
Epoch 2/200
9/9 [==============================] - 2s 193ms/step - loss: 6.7314
Epoch 3/200
9/9 [==============================] - 2s 194ms/step - loss: 5.7389
Epoch 4/200
9/9 [==============================] - 2s 239ms/step - loss: 5.4790
Epoch 5/200
9/9 [==============================] - 2s 251ms/step - loss: 5.3933
Epoch 6/200
9/9 [==============================] - 3s 300ms/step - loss: 5.3202
Epoch 7/200
9/9 [==============================] - 3s 322ms/step - loss: 5.2869
Epoch 8/200
9/9 [==============================] - 2s 260ms/step - loss: 5.2371
Epoch 9/200
9/9 [==============================] - 2s 259ms/step - loss: 5.2173
Epoch 10/200
9/9 [==============================] - 2s 226ms/step - loss: 5.1442
Epoch 11/200
9/9 [==============================] - 3s 329ms/step - loss: 5.1289
Epoch 12/200
9/9 [==============================] - 2s 226ms/step - loss: 5.0658
Epoch 13/200
9/9 [==================

9/9 [==============================] - 2s 192ms/step - loss: 2.7627
Epoch 103/200
9/9 [==============================] - 2s 189ms/step - loss: 2.7453
Epoch 104/200
9/9 [==============================] - 2s 185ms/step - loss: 2.7315
Epoch 105/200
9/9 [==============================] - 2s 180ms/step - loss: 2.7006
Epoch 106/200
9/9 [==============================] - 2s 184ms/step - loss: 2.7064
Epoch 107/200
9/9 [==============================] - 2s 185ms/step - loss: 2.6319
Epoch 108/200
9/9 [==============================] - 2s 178ms/step - loss: 2.6467
Epoch 109/200
9/9 [==============================] - 2s 178ms/step - loss: 2.6230
Epoch 110/200
9/9 [==============================] - 2s 185ms/step - loss: 2.5885
Epoch 111/200
9/9 [==============================] - 2s 182ms/step - loss: 2.5636
Epoch 112/200
9/9 [==============================] - 2s 174ms/step - loss: 2.5724
Epoch 113/200
9/9 [==============================] - 2s 178ms/step - loss: 2.5349
Epoch 114/200
9/9 [===========

In [21]:
encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding , initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.models.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [22]:
def preprocess(input_sentence):
    tokens = input_sentence.lower().split()
    tokens_list = []
    for word in tokens:
        tokens_list.append(tokenizer.word_index[word]) 
    return preprocessing.sequence.pad_sequences([tokens_list] , maxlen=maxlen_questions , padding='post')

In [23]:
tokenizer.word_index['computador']

753

In [24]:
tests = ['oi', 'Você é um computador', 'o que é linguística', 'o que é um opinião', 'até mais']

In [25]:
for i, frase in enumerate(tests):
    frase_processada = preprocess(tests[i])
    states_values = encoder_model.predict(frase_processada)
    empty_target_seq = np.zeros((1 , 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        dec_outputs , h , c = decoder_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += f' {word}'
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros((1 , 1))  
        empty_target_seq[0 , 0] = sampled_word_index
        states_values = [h , c] 
    print(f'Human: {tests[i]}')
    print()
    
    decoded_translation = decoded_translation.split(' end')[0]
    print(f'Bot: {decoded_translation}')
    print('-'*25)

1/1 [==============================] - 0s 24ms/step
Human: oi

Bot:  olá
-------------------------
1/1 [==============================] - 0s 20ms/step
Human: Você é um computador

Bot:  você também
-------------------------
1/1 [==============================] - 0s 20ms/step
Human: o que é linguística

Bot:  a ciência que trata da produção distribuição e consumo da universidade e o linguista de linguística da linguística da aviação registrado publicado e sem artifícios externos
-------------------------
1/1 [==============================] - 0s 21ms/step
Human: o que é um opinião

Bot:  bem é o estudo de moeda nos estados unidos
-------------------------
1/1 [==============================] - 0s 21ms/step
Human: até mais

Bot:  até
-------------------------
